In [1]:
!pip install diffusers transformers accelerate

In [2]:
# Import pustaka yang diperlukan
import torch
from diffusers import DiffusionPipeline
from IPython.display import display, Image
import datetime
import random
import os

# Inisialisasi pipeline SDXL
model_id = "stabilityai/sd-turbo"
pipe = DiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, use_safetensors=True, variant="fp16")

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available. Using GPU.")
    os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
    # Clear the PyTorch cache
    torch.cuda.empty_cache()
    pipe = pipe.to("cuda")
else:
    print("CUDA is not available. Using CPU.")
    pipe = pipe.to("cpu")
    torch.set_num_threads(8)
    torch.set_num_interop_threads(8)


# Define style image
list_images = ["snowy", "rocky", "green", "misty", "foggy", "desert", "tropical", "arctic", "volcanic", "beachy", "forest", "urban"]
list_atmosphere = ["serene", "peaceful", "mysterious", "dreamy", "vibrant", "moody", "dramatic", "calm", "energetic", "whimsical", "melancholic", "adventurous"]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, due to the lack of support for`float16` operations on this device in PyTorch. Please, remove the `torch_dtype=torch.float16` argum

CUDA is not available. Using CPU.


In [3]:
# Fungsi untuk menghasilkan gambar
def generate_image_sdxl(pipe, prompt, num_inference_steps=1, guidance_scale=0.0):
    print(f"Generating image for prompt: {prompt}")

    # Menghasilkan gambar
    with torch.no_grad(): # Menonaktifkan penghitungan gradien untuk inference
        torch.manual_seed(random.randint(0, 1000))  
        image = pipe(prompt, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale, height=512, width=512).images[0]
    
    # Menyimpan gambar
    image_path = f"generated_image_{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}.png"
    image.save(image_path)

    # Clear the PyTorch cache
    torch.cuda.empty_cache()
    
    return image_path

In [ ]:

prompt_template = f"Vector style image of a {random.choice(list_images)} range, flat design, cool colors, and a {random.choice(list_atmosphere)} atmosphere."
image_path = generate_image_sdxl(pipe, prompt_template) # Memasukkan pipe sebagai argumen

display(Image(filename=image_path))

Generating image for prompt: Vector style image of a foggy range, flat design, cool colors, and a calm atmosphere.


  0%|          | 0/1 [00:00<?, ?it/s]